# Introduçao

**Issue relacionada:** [#15](https://github.com/lappis-unb/salic-ml/issues/15)


### Introdução

Um projeto no Salic pode possuir uma planilha orçamentária aprovada, que dentre outras variáveis possui: uma lista de itens, (ex: figurinista, advogado) preço de cada item, unidade de cada item, quantidade de cada item item, valor unitário solicitado e valor unitário aprovado para os projetos que já completaram a fase de admissão. Além disso um projeto possui uma *área* (Artes Cênicas, Música, Museus e Memória por ex.) e um *segmento* (Teatro, Dança, Circo, Mímica, Jogos eletrônicos por ex.).

### Hipótese


**Projetos de mesma área e segmento possuem conjunto de itens similares**

Mais formalmente, se `A` é o conjunto de itens do `projeto_1` e `B` é o conjunto de itens do `projeto_2`, espera-se que `|A intersecção B|` seja diretamente proporcional ao tamanho do conjunto `A união B`.


## Prepararando os dados

**Recarregar automaticamente os módulos**

In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


**Importando módulos python**

In [8]:
import os
import sys
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from salicml.utils.dates import Dates
from salicml.models.projects import Projects

PROJECT_ROOT = os.path.abspath(os.path.join(os.pardir, os.pardir))
DATA_FOLDER = os.path.join(PROJECT_ROOT, 'data', 'raw')

**Carregando os dados da tabela de Projetos**

In [9]:
items_csv_name = 'project_items.csv'
projects_csv = os.path.join(DATA_FOLDER, items_csv_name)

dt_items = pd.read_csv(projects_csv, low_memory=False)
dt_items.head()

,idPronac,PRONAC,Item,Unidade,QtDias,QtItem,nrOcorrencia,VlUnitarioSolicitado,VlTotalSolicitado,VlUnitarioAprovado,VlTotalAprovado,UfItem,MunicipioItem,Etapa,Area,Segmento,DataProjeto
0,114560,93392,Restauração / Conservação,Não Informado,0,0.0,0,NaN,5086.772949,0.0,0.0,BA,Salvador,2 - Produção / Execução,5,5E,2009-07-08 10:08:59
1,136856,113564,Coordenação geral,Verba,60,1.0,1,8000.0,8000.000000,0.0,0.0,BA,Salvador,4 - Custos / Administrativos,2,81,2011-05-04 15:04:45
2,111218,90105,Diretor artístico,Mês,240,1.0,8,15000.0,120000.000000,10000.0,80000.0,SP,São Paulo,2 - Produção / Execução,1,11,2009-02-18 09:09:27
3,111218,90105,Músicos / Intérpretes,Mês,60,2.0,2,6000.0,24000.000000,0.0,0.0,SP,São Paulo,2 - Produção / Execução,1,11,2009-02-18 09:09:27
4,111218,90105,Contra-regra,Mês,240,6.0,8,1600.0,76800.000000,1600.0,76800.0,SP,São Paulo,2 - Produção / Execução,1,11,2009-02-18 09:09:27


### Filtrando os dados

É conhecido que dados muito antigos (de 1992, por exemplo) podem ser inconsistentes, então vamos filtrar os dados a partir de uma data de início.

In [10]:
from datetime import datetime

START_DATE = datetime(day=1, month=1, year=2013)

date_column = 'DataProjeto'
dt_items[date_column] = pd.to_datetime(dt_items[date_column], format = Dates.DATE_INPUT_FORMAT)

dt_items = dt_items[dt_items.loc[:, date_column] >= START_DATE]
print(min(dt_items[date_column]))
dt_items.head()

2013-01-02 10:59:07


,idPronac,PRONAC,Item,Unidade,QtDias,QtItem,nrOcorrencia,VlUnitarioSolicitado,VlTotalSolicitado,VlUnitarioAprovado,VlTotalAprovado,UfItem,MunicipioItem,Etapa,Area,Segmento,DataProjeto
901,178933,146032,Transporte Local / Locação de Automóvel / Comb...,Verba,4,1.0,1,500.0,500.0,500.0,500.0,PB,João Pessoa,2 - Produção / Execução,1,12,2014-04-11 18:50:56
1133,178933,146032,Passagens Aéreas (Descrever os trechos na tela...,Unidade,4,2.0,3,500.0,3000.0,500.0,3000.0,RJ,Rio de Janeiro,2 - Produção / Execução,1,12,2014-04-11 18:50:56
1424,160728,131319,Diretor artístico,Mês,60,2.0,1,1500.0,3000.0,1500.0,3000.0,RJ,Rio de Janeiro,2 - Produção / Execução,1,11,2013-02-14 16:09:08
2760,202095,160506,Direitos autorais,Unidade,365,1.0,1,30000.0,30000.0,30000.0,30000.0,MT,Cuiabá,1 - Pré-Produção / Preparação,6,6F,2016-02-25 17:57:15
4015,202095,160506,Editor,Verba,365,1.0,1,30000.0,30000.0,30000.0,30000.0,MT,Cuiabá,2 - Produção / Execução,6,6F,2016-02-25 17:57:15


# Explorando os dados

Explorar rapidamente os dados a fim de obter insights.

**Dados básicos**

In [11]:
dt_items.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1232336 entries, 901 to 2001712
Data columns (total 17 columns):
idPronac                1232336 non-null int64
PRONAC                  1232336 non-null int64
Item                    1232336 non-null object
Unidade                 1232336 non-null object
QtDias                  1232336 non-null int64
QtItem                  1232336 non-null float64
nrOcorrencia            1232336 non-null int64
VlUnitarioSolicitado    1180077 non-null float64
VlTotalSolicitado       1232336 non-null float64
VlUnitarioAprovado      1232336 non-null float64
VlTotalAprovado         1232336 non-null float64
UfItem                  1232336 non-null object
MunicipioItem           1232336 non-null object
Etapa                   1232336 non-null object
Area                    1232336 non-null int64
Segmento                1232336 non-null object
DataProjeto             1232336 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(5), int64(5), object(6)
me

### Conclusões

#### Contribuições futuras


In [12]:
from salicml.models.project_items import ProjectItems
from salicml.utils.utils import debug

project_items = ProjectItems(dt_items)
pronac_test = 210310

items = project_items.items(pronac_test)
debug('items', items)


items = ['Coreógrafo' 'Elenco protagonista' 'Material efeito especial' 'Impressão'
 'Locação de sala para ensaios' 'Chefe de palco'
 'Locação equipamento de som' 'Produtor' 'Aluguel de instrumentos'
 'Advogado' 'Elenco protagonista' 'Material de consumo'
 'Hospedagem sem alimentação'
 'Transporte Local / Locação de Automóvel / Combustível'
 'Assistente de diretor' 'Microfonista' 'Assistente de cenografia'
 'Maquinista' 'Camareira' 'Correios' 'Carregador' 'Figurinista'
 'Assistente de diretor musical' 'Refeição' 'Contador'
 'Locação equipamento de iluminação' 'Carregador' 'Som' 'Ator/Atriz'
 'Operador de som' 'Locação de equipamentos' 'Chefe de camarim' 'Uniforme'
 'Operador de luz' 'Aluguel de instrumentos'
 'Materiais e equipamentos para montagem' 'Programador' 'Elenco Principal'
 'Passagens Aéreas (Descrever os trechos na tela de deslocamentos)'
 'Material para Primeiros Socorros' 'Locação equipamento de iluminação'
 'Desenhista' 'Preparador Corporal' 'Elenco coadjuvante'
 'Transport